In [1]:
#Pattern recognition 
#imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
#from scipy.special import binom,gamma
from scipy.stats import multivariate_normal
import scipy.special as sps

In [2]:
m = np.array([0,0])
cov = np.array([[1,3.0/5],[3.0/5,2]])

In [9]:
#Bernoulli distribution
#Binary variables
def bern(x,prob):
    a = np.power(prob,x)
    b = np.power(1-prob,1-x)
    return a*b

def bern_ML_mean(D):
    count = sum(D)
    return float(count)/len(D)

#Binomial Distribution
#Number of m observations of x = 1 in a data set of size N
def bino(m,N,prob):
    coef = binom(N,m)
    a = np.power(prob,m)
    b = np.power(1-prob,N-m)
    return coef*a*b

def bino_m_var(N,prob):
    mean = N*prob
    var = N*prob(1-prob)
    return mean,var

#Beta distribution
#conjugate prior for bino dist
def beta(mu,a,b):
    coef = float(gamma(a+b))/(gamma(a)*gamma(b))
    aa = np.power(mu,a-1)
    bb = np.power(1-mu,b-1)
    return coef*aa*bb

def beta_mean_var(a,b):
    mean = float(a)/(a+b)
    var = float(a*b)/((np.power(a+b,2)*(a+b+1)))
    return mean,var

#Gaussian Dist-Most
def gauss(x,mu,sig2):
    coef = 1.0/(np.power(2*np.pi*sig2,.5))
    diff = x-mu
    exp = np.exp((-1.0/(2*sig2))*(np.power(diff,2)))
    return coef*exp

def plot_gauss(m,std2):
    #std = np.power(cov,.5)
    x = np.arange(m-5,m+5,.1)
    y = gauss(x,m,std2)
    plt.plot(x,y)
    plt.show()
    

#mahalanobis dist
def maha_dist(x,mu,cov):
    diff = x-mu
    ret = np.matmul(diff,np.linalg.inv(cov))
    ret = np.matmul(ret,diff.T)
    return ret.diagonal()
    
#multi gauss
def multi_gauss(x,mu,cov):
    D = len(mu)
    a = 1.0/(np.power(2*np.pi,D/2.0))

    b = 1.0/(np.power(np.linalg.det(cov),.5))

    c = np.exp((-.5)*(maha_dist(x,mu,cov)))
    return a*b*c

#plots heat map of multi_gauss
def plot_multi_gauss(x,y,mu,covar):
    nbins = 50
    xx,yy = np.mgrid[x.min():x.max():nbins*1j,y.min():y.max():nbins*1j]
    xxyy = np.vstack([xx.flatten(), yy.flatten()]).T
    zz = multi_gauss(xxyy,mu,covar)
    plt.pcolormesh(xx,yy,zz.reshape(xx.shape))
    plt.show()
    
#returns eignvalues and vectors of matrix
def get_local_multi_gauss(mu,covar):
    return np.linalg.eig(covar)

#return conditional gauss parameters for 2d gauss
#p(xa|xb)
def conditional_gauss(mu,covar,xab,x_is_ab):
    ma = mu[0]
    mb = mu[1]
    covaa = covar[0][0]
    covab = covar[0][1]
    covba = covar[1][0]
    covbb = covar[1][1]

    if (x_is_ab == "b"):
        rtnmu = ma+(((covab)*(np.power(covbb,-1)))*(xab-mb))
        rtncov = covaa-((covab)*(np.power(covbb,-1))*(covba))
    else:
        rtnmu = mb+(((covba)*(np.power(covaa,-1)))*(xab-ma))
        rtncov = covbb-((covba)*(np.power(covaa,-1))*(covab))
    
    return rtnmu,rtncov

#returns marginal gauss parameters for 2d gauss
def marginal_gauss(mu,covar,x_is_ab):
    ma = mu[0]
    mb = mu[1]
    covaa = covar[0][0]
    covbb = covar[1][1]
    if (x_is_ab == "a"):
        return ma,covaa
    else:
        return mb,covbb
    
#given p(x) p(y|x) returns p(y),p(x|y)
def get_marg_gauss(x_m,x_cov,A,b,xab,yx_cov,y):
    AA = np.linalg.inv(x_cov)
    yx_m = np.matmul(A,xab)+b
    L = np.linalg.inv(yx_cov)
    
    y_m = np.matmul(A,x_m)+b
    y_per = L+np.matmul(np.matmul(A,AA),np.transpose(A))
    
    xy_cov = np.transpose(AA+np.matmul(np.matmul(np.transpose(A),L),A))
    xy_m = np.matmul(xy_cov,np.matmul(np.matmul(np.transpose(A),L),(y-b))+np.matmul(AA,x_m))
    
    
    return y_m,y_per,xy_m,xy_cov

#Estiamte maximum likelihood variance
def estimate_gauss_cov(xn,x_m):
    a = 1.0/(len(xn)-1)
    b = np.matmul(xn-x_m,np.transpose(xn-x_m))
    return a*b

#Sequentially estimate maximum likelihood mean
def seq_estimate_gauss(new_x,prev_m,numb_obs):
    N = numb_obs
    new_m = prev_m
    for i in range(0,len(new_x)):
        N = N+1
        new_m = new_m+((1.0/N)*(new_x[i]-new_m))
        
    return new_m,N

#Gaussian mean inference is done using a conjuage prior gaussian
def bayes_infer_gauss_mean(obs,prior_mean,prior_std,std):
    N = len(obs)
    m_ml = (1.0/N)*sum(obs)
    post_mean_a = (float(std)/((N*prior_std)+std))*prior_mean
    post_mean_b = ((N*float(prior_std))/((N*prior_std)+std))*m_ml
    post_mean = post_mean_a+post_mean_b
    
    post_std = (float(prior_std)*std)/(std+(N*prior_std))
    #inv_std = (1.0/prior_std)+(float(N)/std)
    return post_mean,post_std

#Gaussian variance inference is done using a conjugate prior gamma 
#distribution
def bayes_infer_gauss_var(obs,prior_hyp_a,prior_hyp_b,mean):
    N = len(obs)
    post_a = prior_hyp_a+(float(N)/2)
    post_b = prior_hyp_b+((1.0/2)*(np.matmul((obs-mean),np.transpose(obs-mean))))
    return post_a,post_b

#Gaussian variance and mean inference. Using gaussian-gamma function
def bayes_infer_gauss_mean_var(obs,prior_hyp_a,prior_hyp_b,prior_mean,prior_std):
    N = len(obs)
    obs_mean = (1.0/N)*sum(obs)
    obs_var = (1.0/N)*(np.matmul(np.transpose(obs-obs_mean),obs-obs_mean))
    prior_per = 1.0/np.power(prior_std,2)
    
    post_mean = ((prior_per*prior_mean)+(N*obs_mean))/(prior_per+N)
    post_per = prior_per+N
    post_a = prior_hyp_a+(float(N)/2)
    post_b = prior_hyp_b+((1.0/2)*((N*obs_var)+((prior_per*N*np.power(obs_mean-prior_mean,2))/(prior_per+N))))
    post_std = np.power(1.0/post_per,.5)

    return post_mean,post_std,post_a,post_b

#All Gaussian infer example function
def bayes_infer_example(mean,std,numb_obs,obs_sample_size,infer_sample_size):
    obs = []
    for i in range(0,numb_obs):
        obs.append(np.random.normal(mean,std,[obs_sample_size]))
        
    #priors
    m = .1
    s = .1
    a = .1
    b = .1
    mm = .1
    ss = .1
    aa = .1
    bb = .1
    
    for i in range(0,infer_sample_size):
        m,s = bayes_infer_gauss_mean(np.asarray(obs[i]),m,s,std)
    
    print('Infered mean: {0:3f}, Uncertainity:  {1:3f}'.format(m,s))
    
    for i in range(0,infer_sample_size):
        a,b = bayes_infer_gauss_var(np.asarray(obs[i]),a,b,mean)
    
    g = np.random.gamma(a,1.0/b,1000)
    i_std = np.power(1.0/np.mean(g),.5)
    i_std_un = (1.0/(len(g)-1))*np.matmul(np.transpose(g-np.mean(g)),g-np.mean(g))
    i_std_un = np.power(i_std_un,.5)
    print("Post a: {0:3f}, Post b: {1:3f}, Infered std:  {2:3f}, Uncertainity: {3:3f}".format(a,b,i_std,i_std_un))
    
    for i in range(0,infer_sample_size):
        mm,ss,aa,bb = bayes_infer_gauss_mean_var(np.asarray(obs[i]),aa,bb,mm,ss)
    
    gg = np.random.gamma(aa,1.0/bb,1000)
    ii_std = np.power(1.0/np.mean(gg),.5)
    ii_std_un = (1.0/(len(gg)-1))*np.matmul(np.transpose(gg-np.mean(gg)),gg-np.mean(gg))
    ii_std_un = np.power(ii_std_un,.5)
    print('GG-Infered mean: {0:3f}, GG-Uncertainity:  {1:3f}'.format(mm,ss))
    print("GG-Post a: {0:3f}, GG-Post b: {1:3f}, GG-Infered std:  {2:3f}, GG-Uncertainity: {3:3f}".format(aa,bb,ii_std,ii_std_un))

In [10]:
bayes_infer_example(5,.7,1000,10,1000)

Infered mean: 4.994091, Uncertainity:  0.000070
Post a: 5000.100000, Post b: 2399.239670, Infered std:  0.692651, Uncertainity: 0.028570
GG-Infered mean: 4.949027, GG-Uncertainity:  0.009950
GG-Post a: 5000.100000, GG-Post b: 3586.618548, GG-Infered std:  0.847203, GG-Uncertainity: 0.019867
